# Messi Visualizations - Temporada 2025

Visualizaciones enriquecidas con contexto para análisis de Messi.

## 1. Setup

In [ ]:
import pandas as pd
import numpy as np
import os
from mplsoccer.pitch import Pitch
import matplotlib.pyplot as plt
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

BACKGROUND_COLOR = '#313332'
font = 'DejaVu Sans'
SUCCESS_COLOR = '#00FF7F'
FAIL_COLOR = '#FF6B6B'

ORIGIN_COLORS = {
    'Asistido': '#00FF7F',
    'Regate previo': '#00D9FF',
    'Carry individual': '#FFD700',
    'Rebote': '#CCCCCC'
}

## 2. Cargar Datos

In [ ]:
DATA_DIR = './data'

match_folders = sorted([f for f in os.listdir(DATA_DIR) 
                       if os.path.isdir(os.path.join(DATA_DIR, f))])

print(f"Partidos encontrados: {len(match_folders)}")

In [ ]:
def load_all_events():
    all_events = []
    
    for folder in match_folders:
        folder_path = os.path.join(DATA_DIR, folder)
        match_date = folder.split('_')[0]
        
        try:
            df = pd.read_csv(os.path.join(folder_path, 'match_events.csv'))
            df['match_folder'] = folder
            df['match_date'] = match_date
            all_events.append(df)
        except Exception as e:
            print(f"Error en {folder}: {e}")
    
    return pd.concat(all_events, ignore_index=True)

print("Cargando eventos...")
df_events = load_all_events()
df_messi = df_events[df_events['player'] == 'Lionel Messi'].copy()

print(f"Total eventos: {len(df_events):,}")
print(f"Eventos de Messi: {len(df_messi):,}")

## 3. Funciones de Preparación

In [ ]:
def get_previous_events(df, event_idx, window=5):
    event = df.iloc[event_idx]
    team = event['team']
    possession_id = event.get('possession_id')
    
    prev = df[
        (df.index < event_idx) &
        (df['team'] == team) &
        (df['possession_id'] == possession_id)
    ].tail(window)
    
    return prev

In [ ]:
def enrich_assists(df_messi, df_events):
    assists = df_messi[df_messi.get('is_assist', False) == True].copy()
    
    assists['had_dribble_before'] = False
    assists['sequence_length'] = 0
    assists['sequence_duration'] = 0.0
    assists['had_progressive_carry'] = False
    assists['context_type'] = 'Otros'
    
    for idx in assists.index:
        prev = get_previous_events(df_events, idx, window=5)
        
        if len(prev) > 0:
            assists.at[idx, 'sequence_length'] = len(prev)
            assists.at[idx, 'sequence_duration'] = (assists.at[idx, 'minute'] - prev.iloc[0]['minute']) * 60
            
            if any(prev['event_type'] == 'TakeOn'):
                if any((prev['event_type'] == 'TakeOn') & (prev['outcome_type'] == 'Successful')):
                    assists.at[idx, 'had_dribble_before'] = True
                    assists.at[idx, 'context_type'] = 'Con regate'
            
            if any(prev['event_type'] == 'Carry'):
                carries = prev[prev['event_type'] == 'Carry']
                if any(carries.get('is_progressive', False)):
                    assists.at[idx, 'had_progressive_carry'] = True
            
            if assists.at[idx, 'sequence_length'] > 5:
                assists.at[idx, 'context_type'] = 'Secuencia larga'
            elif assists.at[idx, 'sequence_duration'] < 10:
                assists.at[idx, 'context_type'] = 'Contraataque'
    
    return assists

assists_enriched = enrich_assists(df_messi, df_events)
print(f"Asistencias: {len(assists_enriched)}")

In [ ]:
def classify_goal_origin(goal_event, df_events):
    prev = get_previous_events(df_events, goal_event.name, window=3)
    
    if len(prev) > 0:
        if any(prev['event_type'] == 'Pass'):
            last_pass = prev[prev['event_type'] == 'Pass'].iloc[-1]
            if last_pass['player'] != goal_event['player']:
                return 'Asistido'
        
        if any(prev['event_type'] == 'TakeOn'):
            takeons = prev[prev['event_type'] == 'TakeOn']
            if any(takeons['outcome_type'] == 'Successful'):
                return 'Regate previo'
        
        if any(prev['event_type'] == 'Carry'):
            last_carry = prev[prev['event_type'] == 'Carry'].iloc[-1]
            if last_carry.get('pass_distance', 0) > 15:
                return 'Carry individual'
    
    return 'Rebote'

goals_messi = df_messi[
    (df_messi.get('is_goal', False)) | 
    (df_messi['event_type'].str.contains('Goal', case=False, na=False))
].copy()

goals_messi['origin_type'] = goals_messi.apply(
    lambda row: classify_goal_origin(row, df_events), axis=1
)

print(f"Goles de Messi: {len(goals_messi)}")
print(goals_messi['origin_type'].value_counts())

In [ ]:
dribbles_messi = df_messi[df_messi['event_type'] == 'TakeOn'].copy()
print(f"Regates: {len(dribbles_messi)}")
print(f"Exitosos: {(dribbles_messi['outcome_type'] == 'Successful').sum()}")

## 4. Guardar CSVs

In [ ]:
assists_enriched.to_csv('./outputs/messi_assists_enriched.csv', index=False)
goals_messi.to_csv('./outputs/messi_goals_categorized.csv', index=False)
dribbles_messi.to_csv('./outputs/messi_dribbles.csv', index=False)

print("CSVs guardados")

## 5. Viz 1: Pass Map de Asistencias

In [ ]:
assists = assists_enriched[
    assists_enriched['x'].notna() & 
    assists_enriched['end_x'].notna()
].copy()

pitch = Pitch(pitch_color=BACKGROUND_COLOR, line_color='white', linewidth=2, pitch_type='opta')
fig, ax = pitch.draw(figsize=(16, 10))

for _, assist in assists.iterrows():
    context = assist['context_type']
    xg = assist.get('xg', 0.1)
    
    color = {
        'Con regate': SUCCESS_COLOR,
        'Secuencia larga': '#00D9FF',
        'Contraataque': '#FFD700'
    }.get(context, 'white')
    
    width = 2 + (xg * 4)
    
    pitch.arrows(
        assist['x'], assist['y'],
        assist['end_x'], assist['end_y'],
        color=color, alpha=0.6, width=width,
        headwidth=6, headlength=6, ax=ax, zorder=2
    )

fig.set_facecolor(BACKGROUND_COLOR)

fig.text(0.18, 1.03, "Messi - Assists Context Map", fontweight="bold", fontsize=20, color='w', fontfamily=font)
fig.text(0.18, 1, "Inter Miami", fontweight="regular", fontsize=16, color='w', fontfamily=font)
fig.text(0.18, 0.975, "MLS 2025", fontweight="regular", fontsize=12, color='w', fontfamily=font)

fig.text(0.8, 1.03, "Assists:", fontweight="bold", fontsize=12, color='w', fontfamily=font)
fig.text(0.88, 1.03, f"{len(assists)}", fontweight="regular", fontsize=12, color='w', fontfamily=font)

fig.text(0.085, -0.01, "Created by Jaime Oriol", fontweight='bold', fontsize=16, color="white", fontfamily=font)

plt.savefig('./outputs/plots/01_assists_context_map.png', dpi=300, bbox_inches='tight', facecolor=BACKGROUND_COLOR)
print("Viz 1 guardada")
plt.show()

## 6. Viz 2: Shot Map Goles Asistidos

In [ ]:
def get_assisted_shots(df_messi, df_events):
    assists = df_messi[df_messi.get('is_assist', False) == True].copy()
    assisted_shots = []

    for idx in assists.index:
        assist = assists.loc[idx]
        team = assist['team']
        possession_id = assist.get('possession_id')

        next_events = df_events[
            (df_events.index > idx) &
            (df_events['team'] == team) &
            (df_events['possession_id'] == possession_id)
        ].head(3)

        for _, event in next_events.iterrows():
            if event.get('is_shot', False) or 'Shot' in str(event.get('event_type', '')):
                shot_data = {
                    'x_assist': assist['x'],
                    'y_assist': assist['y'],
                    'x_shot': event['x'],
                    'y_shot': event['y'],
                    'xg': event.get('xg', 0.1),
                    'is_goal': event.get('is_goal', False),
                    'body_part': event.get('shot_body_part', 'Foot'),
                    'shooter': event.get('player', 'Unknown')
                }
                assisted_shots.append(shot_data)
                break

    return pd.DataFrame(assisted_shots)

assisted_shots = get_assisted_shots(df_messi, df_events)
assisted_shots.to_csv('./outputs/messi_assisted_shots.csv', index=False)
print(f"Shots asistidos: {len(assisted_shots)}")

## 7. Viz 3: Heatmap Regates

## 6. Viz 3: Heatmap Regates

## 8. Viz 4: Shot Map Goles Categorizados

## 7. Viz 4: Shot Map Goles Categorizados

In [ ]:
goals_valid = goals_messi[
    goals_messi['x'].notna() & 
    goals_messi['y'].notna()
].copy()

pitch = Pitch(pitch_color=BACKGROUND_COLOR, line_color='white', linewidth=2, pitch_type='opta')
fig, ax = pitch.draw(figsize=(16, 10))

for _, goal in goals_valid.iterrows():
    xg = goal.get('xg', 0.1)
    origin = goal['origin_type']
    edge_color = ORIGIN_COLORS.get(origin, '#CCCCCC')
    size = 200 + (xg * 600)
    
    pitch.scatter(
        goal['x'], goal['y'],
        s=size, c='red', marker='o',
        edgecolors=edge_color, linewidths=3,
        alpha=0.9, zorder=3, ax=ax
    )

fig.set_facecolor(BACKGROUND_COLOR)

fig.text(0.18, 1.03, "Messi - Goals by Origin", fontweight="bold", fontsize=20, color='w', fontfamily=font)
fig.text(0.18, 1, "Inter Miami", fontweight="regular", fontsize=16, color='w', fontfamily=font)
fig.text(0.18, 0.975, "MLS 2025", fontweight="regular", fontsize=12, color='w', fontfamily=font)

fig.text(0.8, 1.03, "Goals:", fontweight="bold", fontsize=12, color='w', fontfamily=font)
fig.text(0.88, 1.03, f"{len(goals_valid)}", fontweight="regular", fontsize=12, color='w', fontfamily=font)

fig.text(0.085, -0.01, "Created by Jaime Oriol", fontweight='bold', fontsize=16, color="white", fontfamily=font)

plt.savefig('./outputs/plots/04_goals_categorized.png', dpi=300, bbox_inches='tight', facecolor=BACKGROUND_COLOR)
print("Viz 4 guardada")
plt.show()